In [9]:
# Dependencies
import pandas as pd
import json
import csv
import requests

In [10]:
def create_url(base_url, tables, geo):
    return f"{base_url}?get={tables}&for={geo}"

HOST = "https://api.census.gov/data"
year = "2018"
dataset = "acs/acs5"
base_url = "/".join([HOST, year, dataset])
tables = {"B19301_001E":"Income per capita",
            "B01002_002E":"Median Age Male",
            "B01002_003E":"Median Age Female",
            "B15003_022E":"Bachelor's degree > 25", 
            "B15003_001E":"Total Education",# Will need to divide by total pop >25 B15003_001E
            "B25077_001E":"Median Home Value",
            "B08301_010E":"Public transportation",
            "B08301_001E":"Total transportation"} # Will need to divide by total B08301_001E

table_str = ",".join(tables.keys())
geo = "zip%20code%20tabulation%20area:*"

url = create_url(base_url,table_str,geo)
r = requests.get(url)

# Create a list of human-readable text
col_names = list(tables.values())
col_names.append("Zip Code")

df_CEN = pd.DataFrame(columns = col_names, data = r.json()[1:])

# Set zip code to index
df_CEN.set_index("Zip Code",inplace=True)

# Convert columns to a numeric data types
df_CEN = df_CEN.apply(pd.to_numeric,errors="coerce")

# Convert education and transportation to a percentage of the population
df_CEN["Bachelor's degree > 25"] = df_CEN["Bachelor's degree > 25"]/df_CEN["Total Education"]
df_CEN["Public transportation"] = df_CEN["Public transportation"]/df_CEN["Total transportation"]
df_CEN.drop(["Total Education","Total transportation"],axis=1, inplace=True)
df_CEN.head()

,Income per capita,Median Age Male,Median Age Female,Bachelor's degree > 25,Median Home Value,Public transportation
Zip Code,,,,,,
00601,6999.0,39.5,41.3,0.150448,86200,0.000000
00602,9277.0,41.9,42.8,0.134690,86300,0.002796
00603,11307.0,39.1,42.9,0.171792,122400,0.002939
00606,5943.0,43.2,43.4,0.068866,91600,0.000000
00610,10220.0,40.4,44.2,0.130349,88600,0.002741


In [3]:
#df_CEN.to_csv("Resources/census.csv")

In [26]:
#Filepaths
RE = "https://techhubml.s3.amazonaws.com/real_estate.csv"
CENSUS = "https://techhubml.s3.amazonaws.com/census.csv"

#File paths to DF
#REM to DF
df_RE=pd.read_csv(RE, encoding="ISO-8859-1", dtype=str)
df_CEN=pd.read_csv(CENSUS, encoding="ISO-8859-1", dtype=str)

#View DFs
df_RE
# df_CEN

#View Dtypes
#print(df_RE.dtypes)
# print(df_CEN.dtypes)


,Zip Code,State,City,Metro,CountyName,Average RE Prices
0,00501,NY,Holtsville,New York-Newark-Jersey City,Suffolk County,128202.9167
1,00602,AL,Auburn,Auburn-Opelika,Lee County,114105.25
2,00605,AL,Aguadilla,Dothan,Houston County,78791.83333
3,00606,MO,Neosho,Joplin,Newton County,59377.75
4,00610,KY,Anasco,NaN,Pike County,108219.5833
...,...,...,...,...,...,...
30362,99827,AK,Haines,NaN,Haines Borough,211487
30363,99833,AK,Petersburg,NaN,Petersburg Borough,254642.9167
30364,99835,AK,Sitka,NaN,Sitka Borough,402391.3333
30365,99901,AK,Ketchikan,Ketchikan,Ketchikan Gateway Borough,294448.8333


In [27]:
#Merge DFs
merged_df=pd.merge(df_RE,df_CEN)
merged_df

,Zip Code,State,City,Metro,CountyName,Average RE Prices,Income per capita,Median Age Male,Median Age Female,Bachelor's degree > 25,Median Home Value,Public transportation
0,00602,AL,Auburn,Auburn-Opelika,Lee County,114105.25,9277.0,41.9,42.8,0.1346904527379519,86300,0.0027962250961202375
1,00606,MO,Neosho,Joplin,Newton County,59377.75,5943.0,43.2,43.4,0.06886625332152348,91600,0.0
2,00610,KY,Anasco,NaN,Pike County,108219.5833,10220.0,40.4,44.2,0.1303485702358589,88600,0.0027410320581575495
3,00617,AL,Imbery,Enterprise,Coffee County,101022,9122.0,39.2,41.0,0.15722396714182169,106100,0.005153723120668207
4,00693,PA,Greensburg,Pittsburgh,Westmoreland County,330761.5,10536.0,39.2,41.9,0.1480643208674369,107300,0.00675017397355602
...,...,...,...,...,...,...,...,...,...,...,...,...
29733,99827,AK,Haines,NaN,Haines Borough,211487,33387.0,45.2,45.8,0.22952529994783516,256200,0.002307692307692308
29734,99833,AK,Petersburg,NaN,Petersburg Borough,254642.9167,33227.0,39.3,42.7,0.21967213114754097,213100,0.009210526315789473
29735,99835,AK,Sitka,NaN,Sitka Borough,402391.3333,38423.0,37.2,40.5,0.20449365027678282,349300,0.010178117048346057
29736,99901,AK,Ketchikan,Ketchikan,Ketchikan Gateway Borough,294448.8333,34886.0,39.4,39.9,0.1701417848206839,272300,0.03564781675017895


In [28]:
df_rename=merged_df.rename(columns={'Zip Code':"zipcode"})
df=df_rename.drop(columns=['Metro','Median Home Value'])
final_df=df.set_index('zipcode')

In [29]:
#Change dtypes to their corresponding types
#merged_df.dtypes
final_df=df.astype({'Average RE Prices': 'float64','Income per capita': 'float64','Median Age Male': 'float64','Median Age Female': 'float64',"Bachelor's degree > 25": 'float64',"Public transportation": 'float64'})
final_df.dtypes

zipcode                    object
State                      object
City                       object
CountyName                 object
Average RE Prices         float64
Income per capita         float64
Median Age Male           float64
Median Age Female         float64
Bachelor's degree > 25    float64
Public transportation     float64
dtype: object

In [30]:
#View Final DF
final_df

,zipcode,State,City,CountyName,Average RE Prices,Income per capita,Median Age Male,Median Age Female,Bachelor's degree > 25,Public transportation
0,00602,AL,Auburn,Lee County,114105.2500,9277.0,41.9,42.8,0.134690,0.002796
1,00606,MO,Neosho,Newton County,59377.7500,5943.0,43.2,43.4,0.068866,0.000000
2,00610,KY,Anasco,Pike County,108219.5833,10220.0,40.4,44.2,0.130349,0.002741
3,00617,AL,Imbery,Coffee County,101022.0000,9122.0,39.2,41.0,0.157224,0.005154
4,00693,PA,Greensburg,Westmoreland County,330761.5000,10536.0,39.2,41.9,0.148064,0.006750
...,...,...,...,...,...,...,...,...,...,...
29733,99827,AK,Haines,Haines Borough,211487.0000,33387.0,45.2,45.8,0.229525,0.002308
29734,99833,AK,Petersburg,Petersburg Borough,254642.9167,33227.0,39.3,42.7,0.219672,0.009211
29735,99835,AK,Sitka,Sitka Borough,402391.3333,38423.0,37.2,40.5,0.204494,0.010178
29736,99901,AK,Ketchikan,Ketchikan Gateway Borough,294448.8333,34886.0,39.4,39.9,0.170142,0.035648


In [51]:
#Create CSV
final_df.to_csv("Master.csv",index=False)

In [ ]:
#Check for errors
# df_CHECK=pd.read_csv("Master.csv", encoding="ISO-8859-1", converters={'zipcode': lambda x: str(x)})
# df_CHECK.set_index('zipcode')